In [0]:
# Your ADLS Gen2 storage account
storage_account = "ngtv"

# Your Service Principal parameters
client_id = "0d81a2cb-800b-44be-8efb-267714adfd17"
tenant_id = "79ff955e-08c4-4021-8710-456ab6cd7149"
client_secret = "IH-8Q~eHq1IEMDdvtxOg5kPGw1VTVioTWm5WMb52"


# -------------------------------------------------------------
# Set Spark configs for OAuth2 access
# -------------------------------------------------------------
spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net",
    "OAuth"
)

spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
)

spark.conf.set(
    f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net",
    client_id
)

spark.conf.set(
    f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net",
    client_secret
)

spark.conf.set(
    f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
    f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
)


In [0]:
display(dbutils.fs.ls("abfss://landing@ngtv.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://landing@ngtv.dfs.core.windows.net/Error_disable_logs_1_49.log,Error_disable_logs_1_49.log,27979,1763927050000
abfss://landing@ngtv.dfs.core.windows.net/Error_disable_logs_1_50.log,Error_disable_logs_1_50.log,27919,1763927050000
abfss://landing@ngtv.dfs.core.windows.net/Erroring_links_logs_1_49.log,Erroring_links_logs_1_49.log,33815,1763927050000
abfss://landing@ngtv.dfs.core.windows.net/Erroring_links_logs_1_50.log,Erroring_links_logs_1_50.log,33817,1763927050000
abfss://landing@ngtv.dfs.core.windows.net/Metric_logs_script.log,Metric_logs_script.log,34848,1763927050000
abfss://landing@ngtv.dfs.core.windows.net/data_9b953df9-7b52-49f6-b7de-eeea5575b2ba_21d4468c-9925-4fc9-855a-63b10ab7e823.txt,data_9b953df9-7b52-49f6-b7de-eeea5575b2ba_21d4468c-9925-4fc9-855a-63b10ab7e823.txt,54747,1765372996000
abfss://landing@ngtv.dfs.core.windows.net/data_be3be4a6-1741-4a7d-9b89-9ed255333cbd_f3ce7c0f-ac94-4961-a419-27535fe850f9.txt,data_be3be4a6-1741-4a7d-9b89-9ed255333cbd_f3ce7c0f-ac94-4961-a419-27535fe850f9.txt,54747,1765372643000
abfss://landing@ngtv.dfs.core.windows.net/data_d211c739-c682-480e-b47a-606b74440104_53bcac95-0bcd-4227-a5c0-51ab46b7ee9c.txt,data_d211c739-c682-480e-b47a-606b74440104_53bcac95-0bcd-4227-a5c0-51ab46b7ee9c.txt,54747,1765372757000
abfss://landing@ngtv.dfs.core.windows.net/data_dfba8136-fa1e-4a71-836a-ea820c20bb5b_98fbf723-e823-4236-a915-ca37bdf6476e.txt,data_dfba8136-fa1e-4a71-836a-ea820c20bb5b_98fbf723-e823-4236-a915-ca37bdf6476e.txt,54747,1765372879000


In [0]:
df_49_50_error_disable_logs_merge = spark.read.text(
    "abfss://landing@ngtv.dfs.core.windows.net/Error_disable_logs_1_*.log"
)

df_49_50_error_disable_logs_merge.show(600, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                               |
+----------------------------------------------------------------------------------------------------------------------------------------------------+
|[k1405780@winn-dmzhop-1 ~]$ bulkcmd.pl --class "NGTV - RHE" -f  --cmd "show int te 1/49 | i protocol" --tag 1                                       |
|                                                                                                                                                    |
|To avoid being asked for your password every time, run                                                                                              |
|  . /rizzo/tools/UserScripts/setup_tacacs_env                                                

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Make sure we start with the correct dataframe
df = df_49_50_error_disable_logs_merge

# Order preservation
df = df.withColumn("line_id", F.monotonically_increasing_id())

# Identify line types
is_run_start = F.col("value").rlike(r"RUN START ON ([A-Za-z0-9._-]+)")
is_success   = F.col("value").rlike(r"^[A-Za-z0-9._-]+,TenGigabit")
is_error     = F.col("value").startswith("ERROR:")

# Extract device for RUN START rows
device_from_run = F.regexp_extract("value", r"RUN START ON ([A-Za-z0-9._-]+)", 1)

# Device from success output
device_from_success = F.regexp_extract("value", r"^([^,]+),", 1)

# Interface/status extraction
interface = F.regexp_extract("value", r"^[^,]+,([^,]+)", 1)
status    = F.regexp_extract("value", r"^[^,]+,[^,]+,(.*)", 1)

# Build device candidate
df = df.withColumn(
    "device_candidate",
    F.when(is_run_start, device_from_run)
     .when(is_success, device_from_success)
)

# Forward-fill device for error rows
w = Window.orderBy("line_id").rowsBetween(Window.unboundedPreceding, 0)

df = df.withColumn(
    "device",
    F.last("device_candidate", ignorenulls=True).over(w)
)

# Final record creation
df = df.withColumn(
    "interface_final",
    F.when(is_success, interface)
).withColumn(
    "status_final",
    F.when(is_success, status)
).withColumn(
    "error_final",
    F.when(is_error, F.col("value"))
)

# Keep only meaningful rows
result = df.filter(
    is_success | is_error
).select(
    "device", "interface_final", "status_final", "error_final"
).orderBy("line_id")

result.show(200, truncate=True)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+--------------------+--------------------+--------------------+
|         device|     interface_final|        status_final|         error_final|
+---------------+--------------------+--------------------+--------------------+
| ando-dtvrhe-1a|TenGigabitEtherne...| line protocol is...|                NULL|
| ando-dtvrhe-1b|TenGigabitEtherne...| line protocol is...|                NULL|
| asfd-dtvrhe-2a|                NULL|                NULL|ERROR: ERROR: 'sh...|
| asfd-dtvrhe-2b|                NULL|                NULL|ERROR: ERROR: 'sh...|
| aztw-dtvrhe-1a|TenGigabitEtherne...| line protocol is...|                NULL|
| aztw-dtvrhe-1b|TenGigabitEtherne...| line protocol is...|                NULL|
| bagu-dtvrhe-1a|                NULL|                NULL|ERROR: ERROR: 'sh...|
| bagu-dtvrhe-1b|                NULL|                NULL|ERROR: ERROR: 'sh...|
| barn-dtvrhe-1a|                NULL|                NULL|ERROR: ERROR: 'sh...|
| barn-dtvrhe-1b|           

In [0]:
#df_49_50_error_disable_logs_merge.printSchema()

In [0]:
#result.write.format("delta").mode("overwrite").save(
#    "abfss://bronze@ngtv.dfs.core.windows.net/df_49_50_error_disable_logs_merge"
#)


In [0]:
bronze_path = "abfss://bronze@ngtv.dfs.core.windows.net/"


In [0]:
result.write.format("delta").mode("overwrite").save(
    bronze_path + "df_49_50_error_disable_logs_merge"
)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df_erroring_link_merge = spark.read.text(
    "abfss://landing@ngtv.dfs.core.windows.net/Erroring_links_logs_1_*.log"
)

df_erroring_link_merge.show(600, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                             |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|[k1405780@winn-dmzhop-1 ~]$ bulkcmd.pl --class "NGTV - RHE" -f  --cmd "show int te 1/50 | i errors" --tag 1                                       |
|                                                                                                                                                  |
|To avoid being asked for your password every time, run                                                                                            |
|  . /rizzo/tools/UserScripts/setup_tacacs_env                                                            

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = df_erroring_link_merge

# Add ordering column
df = df.withColumn("line_id", F.monotonically_increasing_id())

# Identify types
is_run_start = F.col("value").rlike(r"RUN START ON ([A-Za-z0-9._-]+)")
is_success   = F.col("value").rlike(r"^[A-Za-z0-9._-]+,")
is_error     = F.col("value").startswith("ERROR:")

# Extract device names
device_from_run = F.regexp_extract("value", r"RUN START ON ([A-Za-z0-9._-]+)", 1)
device_from_success = F.regexp_extract("value", r"^([^,]+),", 1)

df = df.withColumn(
    "device_candidate",
    F.when(is_run_start, device_from_run)
     .when(is_success, device_from_success)
)

# Forward fill the device name
w = Window.orderBy("line_id").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn(
    "device",
    F.last("device_candidate", ignorenulls=True).over(w)
)

# Metrics extraction
metric_regex = {
    "input_errors":      r"(\d+)\s+input errors",
    "crc":               r"(\d+)\s+CRC",
    "frame":             r"(\d+)\s+frame",
    "overrun":           r"(\d+)\s+overrun",
    "ignored":           r"(\d+)\s+ignored",
    "output_errors":     r"(\d+)\s+output errors",
    "collisions":        r"(\d+)\s+collisions",
    "interface_resets":  r"(\d+)\s+interface resets",
}

for col_name, regex in metric_regex.items():
    df = df.withColumn(
        col_name,
        F.when(is_success, F.regexp_extract("value", regex, 1).cast("int"))
    )

# Extract error message
df = df.withColumn(
    "error_message",
    F.when(is_error, F.col("value"))
)

# Filter relevant rows
df_filtered = df.filter(is_success | is_error)

# -----------------------------------------
# FINAL MERGE — one row per device
# -----------------------------------------
result = (
    df_filtered
    .groupBy("device")
    .agg(
        F.max("input_errors").alias("input_errors"),
        F.max("crc").alias("crc"),
        F.max("frame").alias("frame"),
        F.max("overrun").alias("overrun"),
        F.max("ignored").alias("ignored"),
        F.max("output_errors").alias("output_errors"),
        F.max("collisions").alias("collisions"),
        F.max("interface_resets").alias("interface_resets"),
        F.concat_ws(" | ", F.collect_set("error_message")).alias("error_message")
    )
    .withColumn(
        "has_error_message",
        F.when(F.col("error_message") != "", "YES").otherwise("NO")
    )
    .orderBy("device")
)

result.show(200, truncate=True)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+------------+----+-----+-------+-------+-------------+----------+----------------+--------------------+-----------------+
|         device|input_errors| crc|frame|overrun|ignored|output_errors|collisions|interface_resets|       error_message|has_error_message|
+---------------+------------+----+-----+-------+-------+-------------+----------+----------------+--------------------+-----------------+
| ando-dtvrhe-1a|           0|   0|    0|      0|      0|            0|         0|               0|                    |               NO|
| ando-dtvrhe-1b|           0|   0|    0|      0|      0|            0|         0|               0|                    |               NO|
| asfd-dtvrhe-2a|        NULL|NULL| NULL|   NULL|   NULL|         NULL|      NULL|            NULL|ERROR: ERROR: 'sh...|              YES|
| asfd-dtvrhe-2b|        NULL|NULL| NULL|   NULL|   NULL|         NULL|      NULL|            NULL|ERROR: ERROR: 'sh...|              YES|
| aztw-dtvrhe-1a|          

In [0]:
result.write.format("delta").mode("overwrite").save(
    "abfss://bronze@ngtv.dfs.core.windows.net/df_erroring_link_merge"
)

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df_metric_log_script = spark.read.text(
    "abfss://landing@ngtv.dfs.core.windows.net/Metric_logs_script.log"
)

df_metric_log_script.show(200, truncate=False)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|[k1405780@winn-dmzhop-1 ~]$ bulkcmd.pl --class "NGTV - RHE" -f  --cmd "show  run | in metric" --tag 1|
|                                                                                                     |
|To avoid being asked for your password every time, run                                               |
|  . /rizzo/tools/UserScripts/setup_tacacs_env                                                        |
|                                                                                                     |
|Enter your tacacs password:                                                                          |
|Calling Rizzo to get details about devices with class like 'NGT

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = df_metric_log_script.withColumnRenamed("value", "line")

# Create ordering ID
df = df.withColumn("line_id", F.monotonically_increasing_id())

# Detect RUN START & errors
is_run_start = F.col("line").rlike(r"RUN START ON ([A-Za-z0-9._-]+)")
is_error     = F.col("line").startswith("ERROR:")

# Extract device name
df = df.withColumn(
    "device_candidate",
    F.when(is_run_start,
           F.regexp_extract("line", r"RUN START ON ([A-Za-z0-9._-]+)", 1))
)

# Forward fill device
w = Window.orderBy("line_id").rowsBetween(Window.unboundedPreceding, 0)
df = df.withColumn("device", F.last("device_candidate", ignorenulls=True).over(w))


# ----------------------------------------------------
# METRIC EXTRACTION WITH CORRECT PRIORITY
# ----------------------------------------------------

# 1) isis metric <num> (highest priority)
df = df.withColumn(
    "metric_isis",
    F.regexp_extract("line", r"\bisis\s+metric\s+(\d+)\b", 1)
)

# 2) plain metric (NOT isis, NOT set metric, NOT metric-style)
df = df.withColumn(
    "metric_plain",
    F.when(
        (~F.col("line").contains("isis metric")) &
        (~F.col("line").contains("set metric")) &
        (~F.col("line").contains("metric-style")),
        F.regexp_extract("line", r"\bmetric\s+(\d+)\b", 1)
    )
)

# 3) set metric <num> (lowest priority)
df = df.withColumn(
    "metric_set",
    F.regexp_extract("line", r"\bset\s+metric\s+(\d+)\b", 1)
)

# Convert empty strings to null
for c in ["metric_isis", "metric_plain", "metric_set"]:
    df = df.withColumn(c, F.when(F.col(c) == "", None).otherwise(F.col(c)))

# PRIORITY SELECTION
df = df.withColumn(
    "metric_value",
    F.coalesce("metric_isis", "metric_plain", "metric_set")
)

# Extract error message
df = df.withColumn(
    "error_message_line",
    F.when(is_error, F.col("line"))
)

# Keep only rows with metric or error
df_filtered = df.filter(
    F.col("metric_value").isNotNull() | is_error
)

# ----------------------------------------------------
# FINAL AGGREGATION
# ----------------------------------------------------

df_result = (
    df_filtered
    .groupBy("device")
    .agg(
        F.max("metric_value").alias("metric_value"),
        F.concat_ws(" | ", F.collect_set("error_message_line")).alias("error_message")
    )
    # Replace empty or NULL error_message with NULL
    .withColumn(
        "error_message",
        F.when(
            (F.col("error_message").isNull()) |
            (F.trim(F.col("error_message")) == ""),
            None
        ).otherwise(F.col("error_message"))
    )
    # YES only when error_message contains something
    .withColumn(
        "error_hit",
        F.when(F.col("error_message").isNull(), "NO").otherwise("YES")
    )
    .orderBy("device")
)

df_result.show(200, truncate=False)


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+------------+-------------------------------------------------------+---------+
|device         |metric_value|error_message                                          |error_hit|
+---------------+------------+-------------------------------------------------------+---------+
|ando-dtvrhe-1a |10000       |NULL                                                   |NO       |
|ando-dtvrhe-1b |10000       |NULL                                                   |NO       |
|asfd-dtvrhe-2a |20          |NULL                                                   |NO       |
|asfd-dtvrhe-2b |20          |NULL                                                   |NO       |
|aztw-dtvrhe-1a |10000       |NULL                                                   |NO       |
|aztw-dtvrhe-1b |10000       |NULL                                                   |NO       |
|bagu-dtvrhe-1a |10000       |NULL                                                   |NO       |
|bagu-dtvrhe-1b |10000       |

In [0]:
df_result.write.format("delta").mode("overwrite").save(
    "abfss://bronze@ngtv.dfs.core.windows.net/df_metric_log_script"
)

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
%fs ls /path 

com.databricks.backend.daemon.driver.scalakernel.SparkException: No such file or directory /path
  com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:195)
  com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:64)
  com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:175)
  com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
  com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
  com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
  com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
  com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
  scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
  com.dat